In [1]:
import numpy as np 
import pandas as pd
import cplex
from docplex.mp.model import Model
from docplex.mp.solution import SolveSolution
from typing import Union, List

## Constants

In [2]:
E = ['CSC101', 'CSC102', 'CSC103', 'CSC104'] # exams
S = ['Aaron','Bruno','Cell','Dodo','Earl','Frank'] # students
T = ['Dec 1st 9am', 'Dec 1st 12pm', 'Dec 2nd 9am', 'Dec 2nd 12pm', 'Dec 3rd 9am'] # timeslots
R = ['SB1', 'SB2','SB3','SB4'] # rooms
Cp = [1, 2, 3, 5] # capacity of rooms

# course enrolments
He_s = np.random.randint(0,2, (len(E),len(S)))
sumHe_s = np.sum(He_s, axis=1)

In [3]:
He_s, sumHe_s

(array([[1, 0, 1, 1, 0, 0],
        [1, 0, 0, 1, 1, 1],
        [1, 0, 0, 0, 0, 1],
        [1, 1, 0, 1, 1, 1]]),
 array([3, 4, 2, 5]))

## Variables

In [4]:
opt_mod = Model(name='ET MIP')

In [5]:
x = opt_mod.binary_var_matrix(len(E), len(T), name="X_e,t") # whether we use timeslot t for exam e
y = opt_mod.binary_var_matrix(len(E), len(R), name="Y_e,r") # whether we use room r for exam e
# z = opt_mod.binary_var_matrix(len(S), len(E), name="Z_s,e") # whether exam e is allocated to student s 

## Constraints

C1: For all exams, the sum of the allocated timeslots must be equal to 1

$$\sum_{t\in T_c} X_e,_t=1 \;\forall \; e \in E, e \in \{0,1\}$$

C2: For all exams, the sum of the allocated rooms must be equal to 1

$$\sum_{r\in R_e} Y_e,_r = 1 \;\forall \; e \in E $$

C3: For every student and timeslot, the sum of the allocated exams must be less or equal to 1. 
- i.e. students can be at only one exam at a time

C4: For all rooms, the sum of students in a room must be less than the capacity of the room

In [6]:
c1 = opt_mod.add_constraints((sum(x[e, t] for t in range(len(T))) == 1 for e in range(len(E))), names='c1') 

In [229]:
c2 = opt_mod.add_constraints((sum(y[e, r] for r in range(len(R))) >= 1 for e in range(len(E))), names='c2') 

In [8]:
# c3 modified constraint 
for s in range(len(S)):
    for t in range(len(T)):
        cond = sum(x[e,t] * He_s[e,s] for e in range(len(E)))
        if type(cond) != int:
            opt_mod.add_constraint(cond <= 1)

In [9]:
# c4 modified constraint
for r in range(len(R)):
    for t in range(len(T)):
        cond = sum((x[e,t]*y[e,r]) * sumHe_s[e] for e in range(len(E)))
        if type(cond) != int:
            opt_mod.add_constraint(cond <= Cp[r])

In [10]:
# c4 = opt_mod.add_constraints((sum(y[e,r] * sumHe_s[e] for e in range(len(E))) <= Cp[r] for r in range(len(R))), names='c4') 

## Objective Function

$$  minimize\; I_T = \sum_{k=1}^{K_T} \; ceil \; \left[ \sum_{c=1}^{C_K}\; N_c \; * \; (ratio \; students \; to \; invigilators) \right] $$

$$\sum_{r \in R} \; ceil \;  \left[ \sum_{e \in E}y_e,_r ( \sum_{s \in S} H_e,_s ) \; (ratio \; students \; to \; invigilators) \right] * \; cost \; per \; invigilator $$

In [231]:
ratio_of_Inv = 1/60

In [232]:
# OverHeadCostCeil = pulp.LpVariable('OverHeadCostCeil', 0, None, LpInteger)

# prob += OverHeadCostCeil >= OverHeadCost
# prob += TotalModelsCost + OverHeadCostCeil


In [233]:
#OB =  [(sum(y[e,r] * sumHe_s[e] * ratio_of_Inv for e in range(len(E))) for r in range(len(R)))]

up = (sum(1 * sumHe_s[e] * ratio_of_Inv for e in range(len(E))) for r in range(len(R)))
upper_bound=0
for i in up:
    upper_bound += np.ceil(i)
#print(upper_bound)

ceil_obj = []
OB = []
var=[]
qwe = []
sum_sum = []

for r in range(len(R)):
    ceil_obj.append(opt_mod.integer_var(lb=0, ub= upper_bound))
    sum_sum.append(sum(y[e,r] * sumHe_s[e] * ratio_of_Inv for e in range(len(E))))
    opt_mod.add_constraint(ceil_obj[r] >= sum_sum[r])


In [234]:
OB = sum(ceil_obj[r] for r in range(len(R)))
opt_mod.set_objective('min', OB)
opt_mod.print_information()

Model: ET MIP
 - number of variables: 121
   - binary=36, integer=85, continuous=0
 - number of constraints: 124
   - linear=104, quadratic=20
 - parameters: defaults
 - objective: minimize
 - problem type is: MIQCP


In [235]:
#obj_fun =  sum((sum(y[e,r] * sumHe_s[e] * ratio_of_Inv for e in range(len(E))) for r in range(len(R))))
#obj_fun =  sum(OB[r] for r in range(len(R)))

OB = sum(ceil_obj[r] for r in range(len(R)))
opt_mod.set_objective('min', OB)
opt_mod.print_information()

Model: ET MIP
 - number of variables: 121
   - binary=36, integer=85, continuous=0
 - number of constraints: 124
   - linear=104, quadratic=20
 - parameters: defaults
 - objective: minimize
 - problem type is: MIQCP


In [236]:
opt_mod.solve()
opt_mod.print_solution()

objective: 1
  X_e,t_0_3=1
  X_e,t_1_2=1
  X_e,t_2_0=1
  X_e,t_3_4=1
  Y_e,r_0_3=1
  Y_e,r_1_3=1
  Y_e,r_2_3=1
  Y_e,r_3_3=1
  x55=20
  x56=20
  x57=20
  x58=20
  x59=20
  x60=20
  x61=20
  x62=20
  x63=20
  x64=20
  x65=20
  x66=20
  x67=20
  x68=20
  x69=20
  x70=20
  x71=20
  x75=20
  x76=20
  x77=20
  x78=20
  x79=20
  x80=20
  x81=20
  x82=20
  x83=20
  x84=20
  x85=20
  x86=20
  x87=20
  x88=20
  x89=20
  x90=20
  x91=20
  x92=20
  x93=20
  x94=20
  x97=20
  x98=20
  x99=20
  x100=20
  x101=20
  x102=20
  x103=20
  x104=20
  x105=20
  x106=20
  x107=20
  x108=20
  x109=20
  x110=20
  x111=20
  x112=20
  x113=20
  x114=20
  x115=20
  x116=20
  x117=20
  x121=1


## Processing Solution

In [237]:
def process_solution(sol : SolveSolution) -> Union[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Takes a cplex solution and produces a exam schedule
    
    Parameters
    ----------
    sol : SolveSolution
        solution from the solver
    
    Returns
    -------
    final_schedule : pd.DataFrame
        The schedule formatted in readable format for an exam organizer
    
    df_x : pd.DataFrame
        The results for variable x
    
    df_y : pd.DataFrame
        The results for variable y
    """
    # extract solutions as df
    df_x = sol.get_value_df(x).rename(columns={'key_1':'exam','key_2':'timeslot'})
    df_y = sol.get_value_df(y).rename(columns={'key_1':'exam','key_2':'room'})

    # Add rows with the names of courses and timelots
    exam_col = [E[i] for i in range(len(E)) for j in range(len(T))]
    time_col = [T[j] for i in range(len(E)) for j in range(len(T))]
    df_x["EXAM"] = exam_col
    df_x["TIMESLOT"] = time_col

    # Add rows with the names of courses and rooms
    exam_col = [E[i] for i in range(len(E)) for j in range(len(R))]
    room_col = [R[j] for i in range(len(E)) for j in range(len(R))]
    df_y["EXAM"] = exam_col
    df_y["ROOM"] = room_col
    
    # Produce the final schedule
    final_schedule = df_x[df_x["value"]==1].merge(df_y[df_y["value"]==1], on='EXAM', how='left')
    
    return final_schedule, df_x, df_y

In [238]:
def create_enrolment_df(He_s : np.array, S : List[int]) -> pd.DataFrame:
    """
    Creates a dataframe with the students for each exam/course
    """
    exam_student_pairs = []
    for exam in range(len(He_s)):
        students_in_exam_e = []
        for i, student in enumerate(He_s[exam]):
            if student == 1:
                students_in_exam_e.append(S[i])
        exam_student_pairs.append(students_in_exam_e)
        
    enrolment_df = pd.DataFrame(columns=['EXAM','student'])
    enrolment_df['EXAM'] = E
    enrolment_df['student'] = exam_student_pairs

    return enrolment_df

In [239]:
enrolment_df = create_enrolment_df(He_s, S)
enrolment_df

,EXAM,student
0,CSC101,"[Aaron, Cell, Dodo]"
1,CSC102,"[Aaron, Dodo, Earl, Frank]"
2,CSC103,"[Aaron, Frank]"
3,CSC104,"[Aaron, Bruno, Dodo, Earl, Frank]"


In [240]:
sol = opt_mod.solve()
if sol:
    print("Found a solution \n")
    schedule, df_x, df_y = process_solution(sol)
    
    print("Schedule: \n")
    display(schedule.merge(enrolment_df, on='EXAM', how='left'))
else:
    print("Could not find a solution")

Found a solution 

Schedule: 



,exam_x,timeslot,value_x,EXAM,TIMESLOT,exam_y,room,value_y,ROOM,student
0,0,3,1.0,CSC101,Dec 2nd 12pm,0,3,1.0,SB4,"[Aaron, Cell, Dodo]"
1,1,2,1.0,CSC102,Dec 2nd 9am,1,3,1.0,SB4,"[Aaron, Dodo, Earl, Frank]"
2,2,0,1.0,CSC103,Dec 1st 9am,2,3,1.0,SB4,"[Aaron, Frank]"
3,3,4,1.0,CSC104,Dec 3rd 9am,3,3,1.0,SB4,"[Aaron, Bruno, Dodo, Earl, Frank]"
